# Definir client

In [2]:
from groq import Groq

API_KEY_GROQ = "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"
client = Groq(api_key=API_KEY_GROQ)

# Criar classe Agent

Esse codigo é responsável por definir a estrutura do nosso agente, facilitando a interação do modelo com as açoes que o rodeiam

Ele é composto por tres funçoes:

- __init__: recebe o client groq e o promt que orienta as ações que ele realizará

- __call__: é um orquestrador de memória, sendo responsável por armazenar novas informações e manda-la ao agente para uso

- __execute__: é o intermediário entre o client e o modelo, reponável por enviar a memória ao modelo "llama-3.3-70b-versatile"

In [3]:
class Agent:
    
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile", messages=self.messages
        )
        return completion.choices[0].message.content

# Prompt Orientador

É nessa parte do script que está o cerebro do agente ReAct funcionando como o guia do agente.
Nela, são definidas as orientações acerca das funções disponíveis, do fluxo de trabalho e da forma de raciocínio que o modelo deve seguir.
Ela é composta por três partes principais:

1. Introdução:
Apresenta ao agente uma visão geral sobre sua função, explicando o que ele deve fazer e como fazer.

2. Definição de Ferramentas (Tools):
Lista, descreve e exemplifica todas as ações que o agente pode realizar, deixando claro quais ferramentas estão disponíveis e quando devem ser utilizadas.

3. Fluxo de Trabalho:
Mostra, por meio de um exemplo prático, como o agente deve agir em situações reais.
Define padrões para entradas e saídas, orienta o comportamento esperado e exemplifica como aplicar as ferramentas para gerar respostas e executar ações.

In [4]:
system_prompt = """
Você funciona em um ciclo de Pensamento, Ação, PAUSA, Observação.
Ao final do ciclo, você gera uma Resposta.
Use Pensamento para descrever seu raciocínio sobre a pergunta que recebeu.
Use Ação para executar uma das ações disponíveis - então retorne com PAUSA.
Observação será o resultado da ação realizada.

Suas ações disponíveis são:

consultar_tipos_bolo:
ex: consultar_tipos_bolo: chocolate
Retorna um objeto com preco_base_por_kg
Formato: {"nome":"chocolate","preco_base_por_kg":60}

calcular:
ex: calcular: 2 * 60
Executa um cálculo e retorna o número.

Pergunta: Qual é o valor de um bolo de chocolate de 2 quilos?

Pensamento: Preciso listar os sabores necessários.
Ação: consultar_tipos_bolo: chocolate
PAUSA

Você será chamado novamente com isso:

Observação: {"nome":"chocolate", "preco_base_por_kg":60}

Pensamento: Qual o valor de um bolo de 2 quilos?
Ação: calcular: 2 * 60
PAUSA

Você será chamado novamente com isso:

Observação: 120.0

Se você já tiver a resposta, finalize com ela.

Resposta: O valor de um bolo de chocolate de 2 quilos é de 120 reais

Importante: forneça apenas **uma única resposta final** usando o formato "Resposta: ...". Evite repetir a resposta.
""".strip()

# Dicionário básico para acesso de dados

É apenas uma meneira simples de informar o modelo

In [5]:
BOLOS = [
    {"nome": "chocolate",  "preco_base_por_kg": 60},
    {"nome": "baunilha",   "preco_base_por_kg": 55},
    {"nome": "cenoura",    "preco_base_por_kg": 58},
    {"nome": "red velvet", "preco_base_por_kg": 68}
]

# Tools

Nessa sessão são definidas as tools para o agente

- consultar_tipos_bolo: Faz uma verificação lógica, na qual se o nome do dicionário for igual ao nome escolhido pelo usuário ele retorna um dicionário com o nome e valor por quilo do bolo.

- calcular: Avalia uma expressão matemática recebida e retorna o resultado como numero em formato float, usada pelo agente quando ele precisa realizar contas.

- parse_action: O parse_action usa uma função de regex para identificar no texto do modelo qual ação foi solicitada e, a partir disso, converter essa instrução textual na chamada da função correspondente.

- extract_thought: Serve para localizar e extrair o texto do bloco "Pensamento:" da resposta do modelo, isolando apenas o raciocínio que o agente escreveu antes de executar uma ação, dar a resposta ou pausar.

In [6]:
import unicodedata
import math
from typing import Dict, Tuple, Optional
import re, json

def consultar_tipos_bolo(sabor: str) -> Dict:
    for item in BOLOS:
        if item["nome"].lower() == sabor.strip().lower():
            return item
    return {}

def calcular(expr: str) -> float:
    allowed_names = {k: getattr(math, k) for k in dir(math) if not k.startswith("_")}
    return float(eval(expr, {"__builtins__": {}}, allowed_names))

def analisar_acao(texto: str):
    m = re.search(r"A(?:ção|cao)\s*:\s*([a-zA-Z_]+)\s*:\s*(.+)", texto, flags=re.I)
    if not m:
        return None
    return m.group(1).strip(), m.group(2).strip()

def extrair_pensamento(texto: str):
    m = re.search(r"Pensamento\s*:\s*(.*?)(?:\n(?:A(?:ção|cao)|Resposta|PAUSA)\b|$)",
                  texto, flags=re.I|re.S)
    return m.group(1).strip() if m else None

# Função de Loop para iterar etapas

Essa função loop serve para automatizar os processos definidos de realização de tarefas. Colocando em prática o ciclo Pensamento, Ação, PAUSA, Observação até a resposta gerada ser satisfatória.

1) O loop começa recebendo a entrada do usuário (query) e definindo-a como próximo prompt a ser enviado ao modelo.
2) Em seguida, entra em um ciclo que se repete até o número máximo de iterações permitido (neste caso, 10).
3) Cada rodada envia o next_prompt ao modelo e analisa a resposta recebida.
4) Primeiro tenta extrair o “Pensamento” para registrar o raciocínio.
5) Verifica se há uma instrução de ação (Ação: ...) seguida de PAUSA; se houver, executa a ferramenta indicada com o argumento informado.
6) Obtém o resultado (Observação) e prepara um novo prompt com esse valor para a próxima iteração.
7) Se não houver ação, mas a resposta já tiver "Resposta: ...", extrai esse conteúdo e encerra o loop retornando o valor final.
8) Caso contrário, define uma observação e segue para a próxima rodada, até que o modelo forneça a resposta ou o limite de iterações seja atingido.

In [7]:
def loop(query: str, envia_modelo, maximo_iteracoes: int = 10):
    proximo_prompt = query

    for i in range(1, maximo_iteracoes + 1):
        print(f"\n====== Iteração {i} ======")
        print(f"[→ Enviado ao modelo]\n{proximo_prompt}")

        resp = envia_modelo(proximo_prompt)
        print(f"[← Saída do modelo]\n{resp}")
        
        pensamento = extrair_pensamento(resp)
        if pensamento:
            print(f"[Pensamento] {pensamento}")

        analisado = analisar_acao(resp)
        has_pausa = bool(re.search(r"\bPAUSA\b", resp, flags=re.I))
        if analisado and has_pausa:
            tool, arg = analisado
            print(f"[Ação] {tool}({arg})")
            print("[PAUSA] aguardando Observação...")

            if tool == "consultar_tipos_bolo":
                obs = consultar_tipos_bolo(arg)
            elif tool == "calcular":
                obs = calcular(arg)
            else:
                obs = "Ferramenta não encontrada"

            obs_text = json.dumps(obs, ensure_ascii=False) if isinstance(obs, (dict, list)) else str(obs)
            print(f"[Observação] {obs_text}")

            proximo_prompt = f"Observação: {obs_text}"
            continue

        m = re.search(r"Resposta\s*:\s*(.+)", resp, flags=re.I|re.S)
        if m:
            resposta_final = m.group(1).strip()
            print(f"[✅ Resposta final] {resposta_final}")
            return resposta_final

        print("[Aviso] Não detectei Ação/PAUSA nem Resposta. Enviando observação neutra.")
        proximo_prompt = "Observação: (nenhuma ação detectada)"

    return "Não foi possível concluir em tempo."

# Executor

Esse trecho de código é o conector entre o agente e a API da Groq.Ele cria um cliente, mantém um histórico da conversa e define a função que envia mensagens ao modelo, retornando as respostas recebidas dele.

In [9]:
from groq import Groq

API_KEY_GROQ = "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"
client = Groq(api_key=API_KEY_GROQ)

messages = [
    {"role": "system", "content": system_prompt}
]

def envia_modelo(msg: str) -> str:
    messages.append({"role": "user", "content": msg})
    
    resp = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=messages
    )
    
    text = resp.choices[0].message.content
    messages.append({"role": "assistant", "content": text})
    return text

In [10]:
loop("Qual é o valor de um bolo de baunilha de 4 quilos?", envia_modelo)


====== Iteração 1 ======
[→ Enviado ao modelo]
Qual é o valor de um bolo de baunilha de 4 quilos?
[← Saída do modelo]
Pensamento: Preciso saber o preço base por quilo do bolo de baunilha.
Ação: consultar_tipos_bolo: baunilha
PAUSA
[Pensamento] Preciso saber o preço base por quilo do bolo de baunilha.
[Ação] consultar_tipos_bolo(baunilha)
[PAUSA] aguardando Observação...
[Observação] {"nome": "baunilha", "preco_base_por_kg": 55}

====== Iteração 2 ======
[→ Enviado ao modelo]
Observação: {"nome": "baunilha", "preco_base_por_kg": 55}
[← Saída do modelo]
Pensamento: Agora que tenho o preço base por quilo, posso calcular o valor total do bolo de 4 quilos.
Ação: calcular: 4 * 55
PAUSA
[Pensamento] Agora que tenho o preço base por quilo, posso calcular o valor total do bolo de 4 quilos.
[Ação] calcular(4 * 55)
[PAUSA] aguardando Observação...
[Observação] 220.0

====== Iteração 3 ======
[→ Enviado ao modelo]
Observação: 220.0
[← Saída do modelo]
Resposta: O valor de um bolo de baunilha de 4

'O valor de um bolo de baunilha de 4 quilos é de 220 reais'